In [1]:
'''
If you are running locally then 
1. reboot your local machine
2. create an environment called 'colab' using anaconda prompt
if you have a gpu
conda create -n colab python tensorflow-gpu 
if not 
conda create -n colab python tensorflow
3. to install jupyter notebook
conda install jupyter notebook
4. to go to the 'colab' environment
activate colab
5. change file path to locate this notebook and then type 'jupyter notebook'

If you use colab
1. save the data file in your google drive
2. goto colab and start running the code
'''

"\nIf you are running locally then \n1. reboot your local machine\n2. create an environment called 'colab' using anaconda prompt\nif you have a gpu\nconda create -n colab python tensorflow-gpu \nif not \nconda create -n colab python tensorflow\n3. to install jupyter notebook\nconda install jupyter notebook\n4. to go to the 'colab' environment\nactivate colab\n5. change file path to locate this notebook and then type 'jupyter notebook'\n\nIf you use colab\n1. save the data file in your google drive\n2. goto colab and start running the code\n"

In [2]:
#install prominent libraries with specific versions

#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4-tf
#!pip install pandas==0.25.1
#!pip install sklearn==0.21.3
#!pip install matplotlib==3.2.1
#!pip install hyperas
#!pip install hyperopt

In [3]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
#from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, CuDNNLSTM, GRU, Input, Activation, Flatten, BatchNormalization, Reshape,Bidirectional
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, History, ReduceLROnPlateau
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras import regularizers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from matplotlib import style
from tensorflow.python.keras.optimizers import SGD, RMSprop, Adam, Adadelta
from tensorflow.python.keras.utils import np_utils
from sklearn.preprocessing import *
import datetime
from tqdm import tqdm
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from tensorflow.python.keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from tensorflow.python.keras.layers import Reshape, merge, Concatenate, Lambda, Average
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import initializers, regularizers, constraints
from tensorflow.python.keras.initializers import *
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe, rand

Using TensorFlow backend.


In [4]:
#Get library versions
print("tensorflow.__version__ = ", tf.__version__)
# import tensorflow.python.keras
# print("keras.__version__ = ", tensorflow.python.keras.__version__)
import sklearn 
print("sklearn.__version__ = ", sklearn.__version__)
print("numpy.__version__ = ", np.__version__)
print("pandas.__version__ = ", pd.__version__)
import matplotlib
print("matplotlib.__version__ = ", matplotlib.__version__)

tensorflow.__version__ =  1.15.0
sklearn.__version__ =  0.23.1
numpy.__version__ =  1.19.1
pandas.__version__ =  1.1.0
matplotlib.__version__ =  3.2.2


In [5]:
#random seed to generate reproduceable results
from numpy.random import seed
seed(56)
# from tensorflow import set_random_seed
# set_random_seed(56)
random.seed(56)
os.environ['PYTHONHASHSEED']=str(1)
os.environ['TF_KERAS'] = '1'

In [6]:
# create these folders if they do not exist
def build_path(dirName):
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except:
        print("Directory " , dirName ,  " already exists")  

In [7]:
# to check if GPU is getting used locally.....you need to see CPU as well as GPU in the output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12299999786606676982
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7125503181
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18438792381966152159
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
def data():
  i = 7 # the label target - number of days to predict from the input date
  p = 7 #Number of days for target calculated in the data set
  batch_size=512
  CLASSES = 2
  time_steps = 7
  ticker='^GSPC'
    
  # read data
  df=pd.read_csv('../^GSPC_7_days_0_return.csv', index_col = 0, parse_dates = True)
    
  #add additional rolling mean data  
  rm_window =30
  rolling_mean = []
  for a in range(2,rm_window+1):
    df[ticker+'rm_'+str(a)] = df[ticker].rolling(window=rm_window,center=False).mean()
    rolling_mean.append(ticker+'rm_'+str(a))
    
  # create label
  targets=pd.DataFrame([])
  for j in range (1, p+1):
    targets=targets.append(df[ticker+'_{}d_target'.format(j)])
    targets=targets.append(df[ticker+'_{}d'.format(j)])
  targets=targets.T
  df=df.drop(targets.columns, axis=1)
  df=df[rm_window:-i]
  targets=targets[rm_window:-i]
  y=targets['^GSPC_{}d_target'.format(i)]

  #check for NaN and remove
  df.isna().mean().sum()
  y.isna().mean().sum()
  remove_list=[]
  for i in df.isnull().any().iteritems():
    if i[1] == True:
      remove_list.append(i[0])
  df=df.drop(remove_list, axis=1)
  df.isnull().any().mean()
 
  # add percent change
  df=df.pct_change()
  df=df.replace([np.inf, -np.inf],np.nan) 
  df.fillna(0, inplace=True)
  df.isnull().any().mean()
    
  # apply preprocessing 
  x_scaler=RobustScaler()
  x = x_scaler.fit_transform(df)
  # x_pred = x_scaler.fit_transform(x_pred)
  del df
  y=y.values
    
  # apply time steps
  def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
      v = X[i:(i + time_steps)]
      Xs.append(v)
      ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
  x, y = create_dataset(x, y, time_steps)

  # create train and test dataset
  x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=54)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  #y_train = y_train.astype('float32')
  #y_test = y_test.astype('float32')
  y_train = np_utils.to_categorical(y_train, CLASSES, 
                                    #dtype='float32'
                                   )
  y_test = np_utils.to_categorical(y_test, CLASSES, 
                                   #dtype='float32'
                                  )
    
  # adjustment for batch_size
  train_start = x_train.shape[0]%batch_size
  test_start = x_test.shape[0]%batch_size
  x_train = x_train[train_start:]
  y_train = y_train[train_start:]
  x_test = x_test[test_start:]
  y_test = y_test[test_start:]
  embed_size = 60

  return x_train, x_test, y_train, y_test, batch_size, time_steps, embed_size

In [9]:
def create_multi_attention_bidirectional_cudnnlstm_model(x_train, x_test, y_train, y_test, batch_size, time_steps, embed_size):
    
    class LayerNormalization(Layer):
        def __init__(self, eps=1e-6, **kwargs):
            self.eps = eps
            super(LayerNormalization, self).__init__(**kwargs)
        def build(self, input_shape):
            self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],initializer=Ones(), trainable=True)
            self.beta = self.add_weight(name='beta', shape=input_shape[-1:],initializer=Zeros(), trainable=True)
            super(LayerNormalization, self).build(input_shape)
        def call(self, x):
            mean = K.mean(x, axis=-1, keepdims=True)
            std = K.std(x, axis=-1, keepdims=True)
            return self.gamma * (x - mean) / (std + self.eps) + self.beta
        def compute_output_shape(self, input_shape):
            return input_shape

    class ScaledDotProductAttention():
        def __init__(self, d_model, attn_dropout=0.1):
            self.temper = np.sqrt(d_model)
            self.dropout = Dropout(attn_dropout)
        def __call__(self, q, k, v, mask):
            attn = Lambda(lambda x:K.batch_dot(x[0],x[1],axes=[2,2])/self.temper)([q, k])
            if mask is not None:
                mmask = Lambda(lambda x:(-1e+10)*(1-x))(mask)
                attn = Add()([attn, mmask])
            attn = Activation('softmax')(attn)
            attn = self.dropout(attn)
            output = Lambda(lambda x:K.batch_dot(x[0], x[1]))([attn, v])
            return output, attn

    class MultiHeadAttention():
        # mode 0 - big martixes, faster; mode 1 - more clear implementation
        def __init__(self, n_head, d_model, d_k, d_v, dropout, mode=0, use_norm=True):
            self.mode = mode
            self.n_head = n_head
            self.d_k = d_k
            self.d_v = d_v
            self.dropout = dropout
            if mode == 0:
                self.qs_layer = Dense(n_head*d_k, use_bias=False)
                self.ks_layer = Dense(n_head*d_k, use_bias=False)
                self.vs_layer = Dense(n_head*d_v, use_bias=False)
            elif mode == 1:
                self.qs_layers = []
                self.ks_layers = []
                self.vs_layers = []
                for _ in range(n_head):
                    self.qs_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                    self.ks_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                    self.vs_layers.append(TimeDistributed(Dense(d_v, use_bias=False)))
            self.attention = ScaledDotProductAttention(d_model)
            self.layer_norm = LayerNormalization() if use_norm else None
            self.w_o = TimeDistributed(Dense(d_model))

        def __call__(self, q, k, v, mask=None):
            d_k, d_v = self.d_k, self.d_v
            n_head = self.n_head

            if self.mode == 0:
                qs = self.qs_layer(q)  # [batch_size, len_q, n_head*d_k]
                ks = self.ks_layer(k)
                vs = self.vs_layer(v)

                def reshape1(x):
                    s = tf.shape(x)   # [batch_size, len_q, n_head * d_k]
                    x = tf.reshape(x, [s[0], s[1], n_head, d_k])
                    x = tf.transpose(x, [2, 0, 1, 3])  
                    x = tf.reshape(x, [-1, s[1], d_k])  # [n_head * batch_size, len_q, d_k]
                    return x
                qs = Lambda(reshape1)(qs)
                ks = Lambda(reshape1)(ks)
                vs = Lambda(reshape1)(vs)

                if mask is not None:
                    mask = Lambda(lambda x:K.repeat_elements(x, n_head, 0))(mask)
                head, attn = self.attention(qs, ks, vs, mask=mask)  

                def reshape2(x):
                    s = tf.shape(x)   # [n_head * batch_size, len_v, d_v]
                    x = tf.reshape(x, [n_head, -1, s[1], s[2]]) 
                    x = tf.transpose(x, [1, 2, 0, 3])
                    x = tf.reshape(x, [-1, s[1], n_head*d_v])  # [batch_size, len_v, n_head * d_v]
                    return x
                head = Lambda(reshape2)(head)
            elif self.mode == 1:
                heads = []; attns = []
                for i in range(n_head):
                    qs = self.qs_layers[i](q)   
                    ks = self.ks_layers[i](k) 
                    vs = self.vs_layers[i](v) 
                    head, attn = self.attention(qs, ks, vs, mask)
                    heads.append(head); attns.append(attn)
                head = Concatenate()(heads) if n_head > 1 else heads[0]
                attn = Concatenate()(attns) if n_head > 1 else attns[0]

            outputs = self.w_o(head)
            outputs = Dropout(self.dropout)(outputs)
            if not self.layer_norm: return outputs, attn
            # outputs = Add()([outputs, q]) # sl: fix
            return self.layer_norm(outputs), attn

    class PositionwiseFeedForward():
        def __init__(self, d_hid, d_inner_hid, dropout=0.1):
            self.w_1 = Conv1D(d_inner_hid, 1, activation='relu')
            self.w_2 = Conv1D(d_hid, 1)
            self.layer_norm = LayerNormalization()
            self.dropout = Dropout(dropout)
        def __call__(self, x):
            output = self.w_1(x) 
            output = self.w_2(output)
            output = self.dropout(output)
            output = Add()([output, x])
            return self.layer_norm(output)

    class EncoderLayer():
        def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
            self.self_att_layer = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
            self.pos_ffn_layer  = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)
        def __call__(self, enc_input, mask=None):
            output, slf_attn = self.self_att_layer(enc_input, enc_input, enc_input, mask=mask)
            output = self.pos_ffn_layer(output)
            return output, slf_attn


    def GetPosEncodingMatrix(max_len, d_emb):
        pos_enc = np.array([
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)] 
            if pos != 0 else np.zeros(d_emb) 
                for pos in range(max_len)
                ])
        pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2]) # dim 2i
        pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2]) # dim 2i+1
        return pos_enc

    def GetPadMask(q, k):
        ones = K.expand_dims(K.ones_like(q, 'float32'), -1)
        mask = K.cast(K.expand_dims(K.not_equal(k, 0), 1), 'float32')
        mask = K.batch_dot(ones, mask, axes=[2,1])
        return mask

    def GetSubMask(s):
        len_s = tf.shape(s)[1]
        bs = tf.shape(s)[:1]
        mask = K.cumsum(tf.eye(len_s, batch_shape=bs), 1)
        return mask

    class Transformer():
        def __init__(self, len_limit, embedding_matrix, d_model=embed_size, \
                  d_inner_hid=512, n_head=10, d_k=64, d_v=64, layers=2, dropout=0.1, \
                  share_word_emb=False, **kwargs):
            self.name = 'Transformer'
            self.len_limit = len_limit
            self.src_loc_info = False # True # sl: fix later
            self.d_model = d_model
            self.decode_model = None
            d_emb = d_model

            pos_emb = Embedding(len_limit, d_emb, trainable=False, \
                                weights=[GetPosEncodingMatrix(len_limit, d_emb)])

            i_word_emb = Embedding(max_features, d_emb, weights=[embedding_matrix]) # Add Kaggle provided embedding here

            self.encoder = Encoder(d_model, d_inner_hid, n_head, d_k, d_v, layers, dropout, \
                                   word_emb=i_word_emb, pos_emb=pos_emb)


        def get_pos_seq(self, x):
            mask = K.cast(K.not_equal(x, 0), 'int32')
            pos = K.cumsum(K.ones_like(x, 'int32'), 1)
            return pos * mask

        
   
    
    x0 = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]))
    x1 = Bidirectional(CuDNNLSTM(units = {{choice([32, 64, 128])}}, name='x1',  kernel_regularizer=regularizers.l2(0.03), 
              bias_regularizer=regularizers.l1(0.03),
              return_sequences=True, stateful = True))(x0)
    x2 = BatchNormalization(name='x2')(x1)
    x3 = Dropout({{uniform(0, 1)}},name='x3')(x2)
    
    
    if {{choice(['three', 'four','five'])}} == 'three':
        x4, slf_attn = MultiHeadAttention(n_head=3, d_model={{choice([32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(x3, x3, x3)
    elif 'four':
        x4, slf_attn = MultiHeadAttention(n_head=4, d_model={{choice([32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(x3, x3, x3)
    elif 'five':
        x4, slf_attn = MultiHeadAttention(n_head=5, d_model={{choice([32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(x3, x3, x3)   
    
    x5 = Bidirectional(CuDNNLSTM(units = {{choice([32, 64, 128])}},name='x5',  kernel_regularizer=regularizers.l2(0.03), 
              bias_regularizer=regularizers.l1(0.03),
              return_sequences=False,stateful = True))(x4)
    x6 = BatchNormalization(name='x6')(x5)
    x7 = Dropout({{uniform(0, 1)}},name='x7')(x6)
    
    if {{choice(['dense', 'nodense'])}} == 'dense':
    
        x8 = Dense(units = {{choice([8, 16, 32])}}, name='x8', kernel_regularizer=regularizers.l2(0.03), bias_regularizer=regularizers.l1(0.03),
                   activation='relu')(x7)
        x9 = Dropout({{uniform(0, 1)}}, name='x9')(x8)
    
        outp = Dense(2, activation='softmax')(x9)
    else:
        outp = Dense(2, activation='softmax')(x7)
        

    model = Model(inputs=[x0], outputs=outp)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = Adam(lr=0.001))

    model.summary()


    result = model.fit(x_train,y_train,batch_size=batch_size,epochs=100,validation_data=(x_test, y_test), verbose=0)
    
    validation_acc = np.amax(result.history['val_acc']) 

    print('Best validation acc of epoch:', result.history['val_acc'])
    print('Train acc of epoch:', result.history['acc'])
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [11]:
if __name__ == '__main__':
    x_train, x_test, y_train, y_test, batch_size, _ ,_ = data()
    print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    
    best_run, best_model = optim.minimize(model=create_multi_attention_bidirectional_cudnnlstm_model, data=data ,algo=tpe.suggest, max_evals=20,trials=Trials(), notebook_name='5.3_old Multi_Attention_Bidirectional_CuDNNLSTM',rseed=1, verbose=False)
    print("Evaluation of best performing model:")
    #best_model.save("MULTI_ATTENTION_BIDIRECTIONAL_CUDNNLSTM_bestmodel.h5")
    #print(best_model.get_config())
    test_score, test_accuracy = best_model.evaluate(x_test, y_test, batch_size=batch_size)
    print('test_score: ', test_score, ' test_accuracy: ', test_accuracy)
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

(8704, 7, 2251) (8704, 2) (3584, 7, 2251) (3584, 2)
  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From C:\Users\kenneth\Anaconda3\envs\tf115\lib\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"                                                                                                         
________________

__________________________________________________________________________________________________                     
layer_normalization (LayerNorma (512, 7, 128)        256         dropout_1[0][0]                                       
__________________________________________________________________________________________________                     
bidirectional_1 (Bidirectional) (512, 64)            41472       layer_normalization[0][0]                             
__________________________________________________________________________________________________                     
x6 (BatchNormalization)         (512, 64)            256         bidirectional_1[0][0]                                 
__________________________________________________________________________________________________                     
x7 (Dropout)                    (512, 64)            0           x6[0][0]                                              
________________________________________

dense_6 (Dense)                 (512, 7, 256)        65536       x3[0][0]                                              
__________________________________________________________________________________________________                     
lambda_6 (Lambda)               (2048, 7, 64)        0           dense_5[0][0]                                         
__________________________________________________________________________________________________                     
lambda_7 (Lambda)               (2048, 7, 64)        0           dense_6[0][0]                                         
__________________________________________________________________________________________________                     
lambda_9 (Lambda)               (2048, 7, 7)         0           lambda_6[0][0]                                        
                                                                 lambda_7[0][0]                                        
________________________________________

 10%|████▊                                           | 2/20 [03:09<27:45, 92.52s/trial, best loss: -0.7712053656578064]WARNING:tensorflow:Large dropout rate: 0.980758 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
Model: "model_2"                                                                                                       
__________________________________________________________________________________________________                     
Layer (type)                    Output Shape         Param #     Connected to                                          
input_3 (InputLayer)            [(512, 7, 2251)]     0                                                                 
__________________________________________________________________________________________________                     
bidirectional_4 (Bidirectional) (512, 7, 128)        1186304     input_3[0][0]                                         

Best validation acc of epoch:                                                                                          
[0.46763393, 0.5139509, 0.530692, 0.55022323, 0.55050224, 0.5398995, 0.5485491, 0.53376114, 0.5404576, 0.5421317, 0.54938614, 0.55106026, 0.54910713, 0.5452009, 0.5485491, 0.5541295, 0.5558036, 0.55691963, 0.5535714, 0.5541295, 0.5558036, 0.5555245, 0.5555245, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5552455, 0.5

...

Train acc of epoch:                                                                                                    
[0.5674403, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57433367, 0.57

__________________________________________________________________________________________________                     
bidirectional_35 (Bidirectional (512, 64)            25088       layer_normalization_17[0][0]                          
__________________________________________________________________________________________________                     
x6 (BatchNormalization)         (512, 64)            256         bidirectional_35[0][0]                                
__________________________________________________________________________________________________                     
x7 (Dropout)                    (512, 64)            0           x6[0][0]                                              
__________________________________________________________________________________________________                     
dense_89 (Dense)                (512, 2)             130         x7[0][0]                                              
Total params: 1,327,170                 

lambda_109 (Lambda)             (2048, 7, 64)        0           dense_91[0][0]                                        
__________________________________________________________________________________________________                     
lambda_111 (Lambda)             (2048, 7, 7)         0           lambda_108[0][0]                                      
                                                                 lambda_109[0][0]                                      
__________________________________________________________________________________________________                     
activation_18 (Activation)      (2048, 7, 7)         0           lambda_111[0][0]                                      
__________________________________________________________________________________________________                     
dense_92 (Dense)                (512, 7, 256)        65536       x3[0][0]                                              
________________________________________

Model: "model_19"                                                                                                      
__________________________________________________________________________________________________                     
Layer (type)                    Output Shape         Param #     Connected to                                          
input_20 (InputLayer)           [(512, 7, 2251)]     0                                                                 
__________________________________________________________________________________________________                     
bidirectional_38 (Bidirectional (512, 7, 256)        2438144     input_20[0][0]                                        
__________________________________________________________________________________________________                     
x2 (BatchNormalization)         (512, 7, 256)        1024        bidirectional_38[0][0]                                
________________________________________

Train acc of epoch:                                                                                                    
[0.5132123, 0.51953125, 0.5127528, 0.5189568, 0.52079505, 0.52952665, 0.5297564, 0.5279182, 0.53550094, 0.5383732, 0.55204505, 0.5465303, 0.54423255, 0.55388325, 0.5543428, 0.5565257, 0.56732535, 0.5844439, 0.5938649, 0.5934053, 0.6130515, 0.61638325, 0.6261489, 0.6485524, 0.6506204, 0.65326285, 0.66463697, 0.6665901, 0.6738281, 0.6793428, 0.6902574, 0.6889936, 0.6925551, 0.6907169, 0.6894531, 0.6919807, 0.6931296, 0.6972656, 0.6950827, 0.6931296, 0.69485295, 0.71036303, 0.6984145, 0.7100184, 0.7042739, 0.7024357, 0.69795495, 0.70484835, 0.7068015, 0.6971507, 0.7105928, 0.7034697, 0.7068015, 0.70358455, 0.7089844, 0.7027803, 0.706227, 0.704159, 0.70932907, 0.71300554, 0.7038143, 0.70576745, 0.707261, 0.71254593, 0.70668656, 0.7028952, 0.7080653, 0.71392465, 0.7027803, 0.706227, 0.70450366, 0.706227, 0.70668656, 0.70576745, 0.71300554, 0.71714157, 0.70714617, 0.7054228

In [ ]:
best_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint(filepath='5.3 Multi_Attention_Bidirectional_CuDNNLSTM.h5', save_best_only=True, monitor='val_loss', verbose=0, mode='min')
callbacks_list = [checkpoint]

history = best_model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=1000, validation_data=(x_test, y_test), verbose=0, callbacks=callbacks_list)